## Logic to Karnaugh map

[Wiki: Karnaugh map](https://en.wikipedia.org/wiki/Karnaugh_map)
![](https://upload.wikimedia.org/wikipedia/commons/thumb/0/02/K-map_6%2C8%2C9%2C10%2C11%2C12%2C13%2C14_anti-race.svg/440px-K-map_6%2C8%2C9%2C10%2C11%2C12%2C13%2C14_anti-race.svg.png)

In [1]:
from sympy.combinatorics.graycode import GrayCode
from itertools import permutations
import numpy as np

In [2]:
def ReverseBinCode(l):
    # Generate reversed binary code for pyphi
    maxIndex = 2 ** l - 1
    return [np.binary_repr(maxIndex - index, width=l) for index in range(2 ** l)]

## 输入`pyphi`中状态转移矩阵，给出各个变量的卡诺图

In [3]:
# 输入状态转移列表，给出各个量的卡诺图矩阵

def MapToKarnaughMap(_map):
    # Turn map to MapToKarnaugh Maps
    # Generate ReverseBinCode
    index = ReverseBinCode(len(_map[0]))
    #index = np.array([[int(char) for char in strcode] for strcode in index])
    
    dim1 = (int(len(_map[0]) / 2))
    dim2 = (len(_map[0]) - int(len(_map[0]) / 2))
    #print('dim =', dim1, 'x', dim2)
    
    gc1 = GrayCode(dim1)
    gc2 = GrayCode(dim2)
    gd1 = dict(zip(list(gc1.generate_gray()), range(2 ** dim1)))
    gd2 = dict(zip(list(gc2.generate_gray()), range(2 ** dim2)))
    #print(gd1)
    #print(gd2)
    
    _map = np.array(_map).T
    KarnaughMaps = []
    for targetStates in _map:
        #print('-------')
        Karnaugh = np.array(([[0] * (2 ** dim1)]) * (2 ** dim2)).T
        #print(Karnaugh)
        for state, mcode in zip(index, targetStates):
            #print(state, '->', mcode)
            #print('c1=',state[0:dim1])
            #print('c2=',state[dim1:])
            Karnaugh[gd1[state[0:dim1]], gd2[state[dim1:]]] = mcode
        KarnaughMaps += [Karnaugh]
        #print(Karnaugh)
    
    return np.array(KarnaughMaps)

## 输入矩阵，给出Ising模型能量

In [4]:
def IsingEnergy(mat, energy={0:-1, 1:1}):
    # 给出mat的Ising能量
    # Return Ising energy of mat
    mat = np.array(mat)
    # If mat is a list of mat, then return all energy
    if len(mat.shape) == 3:
        return np.array([IsingEnergy(_mat, energy) for _mat in mat])
    # Otherwise return this energy
    x, y = mat.shape
    total_E = 0
    for i in range(x):
        for j in range(y):
            total_E += energy[mat[i][j]] * energy[mat[(i - 1) % x][j]]
            total_E += energy[mat[i][j]] * energy[mat[(i + 1) % x][j]]
            total_E += energy[mat[i][j]] * energy[mat[i][(j - 1) % y]]
            total_E += energy[mat[i][j]] * energy[mat[i][(j + 1) % y]]
    return -total_E

In [57]:
m = [
    [0,0,0],
    [0,0,1],
    [0,1,0],
    [0,1,1],
    [1,0,0],
    [1,0,1],
    [1,1,0],
    [1,1,1]
    ]
MapToKarnaughMap(m)

array([[[1, 1, 1, 1],
        [0, 0, 0, 0]],

       [[1, 1, 0, 0],
        [1, 1, 0, 0]],

       [[1, 0, 0, 1],
        [1, 0, 0, 1]]])

## 对状态名重新标记 Relabel

In [6]:
testorder=permutations(m).__next__()

In [40]:
def code2str(codeList):
    s = ''
    for i in codeList:
        s += str(i)
    return s

def relabel(m0, order):
    length = len(m0[0])
    bcode = ReverseBinCode(length)
    reorderRule = {}
    for originLabel, newLabel in zip(bcode, order):
        reorderRule[originLabel] = newLabel
    #print(reorderRule)
    rule = {}
    for currentState, nextState in zip(bcode, m0):
        #print(reorderRule[code2str(currentState)], 'to', str(nextState))
        rule[code2str(reorderRule[code2str(currentState)])] = reorderRule[code2str(nextState)]
    
    new_m = [rule[startState] for startState in ReverseBinCode(length)]
    
    return np.array(new_m)

In [41]:
print(relabel(m, testorder))

[[0 0 0]
 [0 0 1]
 [0 1 0]
 [0 1 1]
 [1 0 0]
 [1 0 1]
 [1 1 0]
 [1 1 1]]


In [43]:
MapToKarnaughMap(relabel(testM, testOrder))

array([[[0, 0],
        [1, 1]],

       [[0, 1],
        [1, 0]]])

## 计算不同排序的Ising能量

In [54]:
m = [
    [0,0,0],
    [0,0,1],
    [0,1,0],
    [0,1,1],
    [1,0,0],
    [1,0,1],
    [1,1,0],
    [1,1,1]
    ]
energyList = []
maxlen = 50

testorders=permutations(m)

for i, order in zip(range(maxlen), testorders):
    current_m = relabel(m, list(order))
    energy = [IsingEnergy(kmap) for kmap in MapToKarnaughMap(current_m)]
    energyList += energy

print(energyList)

[0, -16, -16, 0, -16, 32, 0, -16, -16, 0, 8, 8, 0, 8, 8, 0, 32, -16, 0, -16, 32, 0, -16, -16, 0, 8, 8, 0, 0, -16, 0, 16, 16, 0, 8, 8, 0, 8, 8, 0, 16, 16, 0, 32, -16, 0, 8, 8, 0, -16, -16, 0, -16, 0, 0, 0, -16, 0, 8, 8, 0, 8, 8, 0, -16, -16, 0, -16, 0, 0, -16, -16, 0, -16, -16, 0, -16, 32, 0, -16, -16, 0, 8, 8, 0, 8, 8, 0, 32, -16, -16, 0, -16, -16, 0, 32, 0, 0, 0, 8, 0, 8, 0, 16, 16, 8, 16, -16, 0, 0, 0, 0, 16, 16, -16, -16, 0, 8, -16, 16, -16, 32, 0, 8, 8, 0, 8, 0, 8, 8, 16, -16, 8, -16, 16, 32, -16, -16, 8, 8, 0, 32, -16, -16, 0, -16, 32, 0, -16, -16]
